In [1]:
import pandas as pd

In [2]:
url_fe = 'https://docs.google.com/spreadsheet/pub?key=0Aul9Ys3cd80fdHNuRG5VeWpfbnU4eVdIWTU3Q0xwSEE&single=TRUE&gid=0&output=csv'
url_ds = 'https://docs.google.com/spreadsheets/d/1cEGQ3eAFKpFBVq1k2mZIy5mBPxC6nBTJHzuSWtZQSVw/export?format=csv'

fe = pd.read_csv(url_fe)
ds = pd.read_csv(url_ds)

In [21]:
def name_processor(df, col_name):

    names = df[col_name]
    names = names.str.lower()
    names = names.str.replace('\d', '')  # remove numeric characters
    names = names.str.replace(',', '')
    names = names.str.replace('[.]', '')

    suffixes = []
    cnt = 0

    for name in names:
        w = str(name).rsplit(' ')[-1]
        if w in ['jr', 'sr', 'ii', 'iii', 'iv']:
            suffixes.append(w)
            names[cnt] = str(names[cnt]).replace(' ' + w, '')
            continue
        else:
            suffixes.append('')
            cnt += 1

    def try_pop(lst, index = -1):
        try:
            return(lst.pop(index))
        except:
            return('')

    names = names.map(lambda x: str(x).rsplit(' '))
    last_name = names.map(lambda x: try_pop(x, index = -1))
    first_name = names.map(lambda x: try_pop(x, index = 0))
    middle_name = names.map(lambda x: str(' '.join(x)))
    
    return(last_name, first_name, middle_name, suffixes)

In [205]:
col_mapping = \
    {'fe': {'name': 'Subject\'s name', \
            'age': 'Subject\'s age', \
            'gender': 'Subject\'s gender', \
            'race': 'Subject\'s race', \
            'hispanic_latin': None, \
            'mental_illness': 'Symptoms of mental illness?', \
            'date': 'Date of injury resulting in death (month/day/year)', \
            'address': 'Location of injury (address)', \
            'city': 'Location of death (city)', \
            'state': 'Location of death (state)', \
            'zip_code': 'Location of death (zip code)', \
            'county': 'Location of death (county)', \
            'agency': 'Agency responsible for death', \
            'source': 'Link to news article or photo of official document', \
            'justified': 'Official disposition of death (justified or other)', \
            'description': 'A brief description of the circumstances surrounding the death' \
           }, \
     'ds': {'name': 'Victim Name', \
            'age': 'Victim\'s Age', \
            'gender': 'Victim\'s Gender', \
            'race': 'Race', \
            'hispanic_latin': 'Hispanic or Latino Origin', \
            'mental_illness': None, \
            'date': 'Date of Incident', \
            'address': None, \
            'city': 'City', \
            'state': 'State', \
            'zip_code': None, \
            'county': 'County', \
            'agency': 'Agency Name', \
            'source': 'Source Link', \
            'justified': 'Was the Shooting Justified?', \
            'description': 'Summary' \
           }
    }

# pd.DataFrame(columns = col_mapping['ds'].keys())
cols_select = list(col_mapping['ds'].values())
cols_keep = []
for col_name in cols_select:
    if col_name != None:
        cols_keep.append(col_name)
x = 'ds'
# eval(x)[cols_keep]



# ds_processed = df_processor('ds', col_mapping['ds'])
# fe_processed = df_processor('fe', col_mapping['fe'])
st = 'ds'
# df_processor(st, col_mapping[st])

length = ds.shape[0]  # number of rows

cols_names = list(col_mapping['ds'].keys())
cols_select = list(col_mapping['ds'].values())
# length = len(ds[cols_names[0]])
cols_keep = []
output = pd.DataFrame(columns = col_mapping['ds'].keys())

cnt = 0
for col in cols_select:
    if col == None:
        col_values = [None] * length
    else:
        col_values = ds[col]
    output[cols_names[cnt]] = col_values
    cnt += 1
    

def shootings_df_process(df_name, col_mappings):
    df = eval(df_name)
    length = df.shape[0]
    
    cols_names = list(col_mappings[df_name].keys())
    cols_select = list(col_mappings[df_name].values())
    # create an empty dataframe as a destination for data
    df_output = pd.DataFrame(columns = col_mapping[df_name].keys())
    
    cnt = 0
    for col in cols_select:
        if col == None:
            col_values = [None] * length
        else:
            col_values = df[col]
        
        df_output[cols_names[cnt]] = col_values
        cnt += 1
        
    return(df_output)

ds_processed = shootings_df_process('ds', col_mapping)
fe_processed = shootings_df_process('fe', col_mapping)

shootings = pd.concat([ds_processed, fe_processed]).reset_index()
last, first, middle, suffix = name_processor(shootings, 'name')
shootings['name_last'] = last
shootings['name_first'] = first
shootings['name_middle'] = middle
shootings['name_suffix'] = suffix
shootings = shootings.drop('name', 1)

In [213]:
# shootings = shootings[shootings['name_last'] != 'nan']
# shootings = shootings[shootings['name_last'] != 'unknown']
# shootings = shootings[shootings['name_last'] != 'police']
# shootings_cnt = shootings.groupby(['name_last', 'name_first', 'state']).count().sort('index', ascending=False)['index']
# # shootings_cnt.head()

shootings['race_black'] = shootings['race'].str.contains('african|black', case = False, na = False)
shootings['race_white'] = shootings['race'].str.contains('white|caucasian', case = False, na = False)
shootings['race_hispanic'] = shootings['race'].str.contains('latin|hispanic', case = False, na = False)
shootings['race_asian'] = shootings['race'].str.match('asian', case = False, na = False)
shootings['race_nativeamer'] = shootings['race'].str.contains('native', case = False, na = False)
# shootings = shootings.drop('race', 1)
shootings['state'] = shootings['state'].map(lambda x: str(x).split(' ')[0])
shootings['mental_illness'] = shootings['mental_illness'].map(lambda x: str(x).lower() == 'yes')
shootings['gender_male'] = shootings['gender'].map(lambda x: str(x).lower() == 'male')


shootings_filt = []

for num in list(range(1,5 + 1)):
    df = shootings[shootings['justified'] == num]
    shootings_filt.append(df)

data = pd.concat(shootings_filt)
shootings['gender_male']

0     True
1     True
2     True
3     True
4     True
5     True
6     True
7     True
8     True
9     True
10    True
11    True
12    True
13    True
14    True
...
9400     True
9401     True
9402     True
9403     True
9404     True
9405     True
9406     True
9407     True
9408    False
9409     True
9410    False
9411     True
9412     True
9413     True
9414     True
Name: gender_male, Length: 9415, dtype: bool

In [379]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.cross_validation import train_test_split
from sklearn.metrics import roc_curve
import matplotlib.pyplot as plt

%matplotlib inline


data = data[['justified','mental_illness','state','gender_male','age','race_black','race_white','race_hispanic','race_asian','race_nativeamer']]

dummies = pd.DataFrame(pd.get_dummies(data['state']))

X = pd.concat([data.drop('state', 1), dummies], axis = 1).dropna()

y = X['justified']
X = X.drop('justified',1)

X_train, X_test, y_train, y_test = train_test_split(X, y)

model_rf = RandomForestClassifier(n_estimators = 25, max_depth = 2500, n_jobs = -1)
model_rf.fit(X_train, y_train)
rf_predict = model_rf.predict(X_test)

model_lr = LogisticRegressionCV(Cs = [0.01,.1, 1, 2.5, 5, 10], cv = 100)
model_lr.fit(X_train, y_train)
lr_predict = model_lr.predict(X_test)

model_gbm = GradientBoostingClassifier(loss = 'deviance', n_estimators = 1000, max_depth = 5)
model_gbm.fit(X_train, y_train)
gbm_predict = model_gbm.predict(X_test)

In [402]:
predictions = pd.DataFrame({'actual': y_test, 'rf_predict':rf_predict, \
                            'lr_predict':lr_predict,'gbm_predict':gbm_predict})
predictions['avg_predict'] = (predictions['lr_predict'] + predictions['rf_predict'] + predictions['gbm_predict']) / 3
predictions['rf_error'] = abs(predictions['actual'] - predictions['rf_predict'])
predictions['lr_error'] = abs(predictions['actual'] - predictions['lr_predict'])
predictions['gbm_error'] = abs(predictions['actual'] - predictions['gbm_predict'])
predictions['avg_error'] = abs(predictions['actual'] - predictions['avg_predict'])
# predictions[['rf_error','lr_error','gbm_error','avg_error']].mean()
# for num in [1,2,3,4,5]:
#     pred_sub = predictions[predictions['actual'] == num]
#     print(num)
#     print(pred_sub[['rf_error','lr_error','gbm_error','avg_error']].mean())

# pred_sub = predictions[predictions['actual'] == 1]
# pred_sub[['rf_error','lr_error','gbm_error','avg_error']].mean()
# predictions.groupby('actual').count()
predictions['avg_predict_rnd'] = predictions['avg_predict'].map(lambda x: round(int(x) , 0))
predictions

,actual,gbm_predict,lr_predict,rf_predict,avg_predict,rf_error,lr_error,gbm_error,avg_error,avg_predict_rnd
1919,3,1,3,3,2.333333,0,0,2,0.6666667,2
1164,5,5,3,5,4.333333,0,2,0,0.6666667,4
1002,1,1,3,1,1.666667,0,2,0,0.6666667,1
1894,4,2,3,4,3,0,1,2,1,3
1571,3,1,3,3,2.333333,0,0,2,0.6666667,2
1733,5,3,3,3,3,2,2,2,2,3
1964,4,5,3,5,4.333333,1,1,1,0.3333333,4
1699,1,1,3,1,1.666667,0,2,0,0.6666667,1
1567,1,3,3,3,3,2,2,2,2,3
766,1,1,1,3,1.666667,2,0,0,0.6666667,1


In [383]:
def get_coefficients(df, model):
    cdf = pd.DataFrame({'Features': df.columns, 'Coef': model.coef_[0]})
    return cdf.sort('Coef', ascending=False)

get_coefficients(X_train, model_lr)[0:4]

,Coef,Features
50,0.841513,UT
3,0.792801,race_black
51,0.753770,VA
30,0.671044,MI
